In [ ]:
!pip freeze | grep scikit-learn

In [ ]:
!python -V

In [ ]:
import pickle
import pandas as pd
import os

In [ ]:
year = 2023
month = 3
input_url = f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet'
output_dir = 'output'
output_file = f'{output_dir}/predictions_{year}-{month:02d}.parquet'

In [ ]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [ ]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [ ]:
input_url

In [ ]:
df = read_data(input_url)

In [ ]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [ ]:
print(f"The standard deviation of the predicted trip duration is {y_pred.std():.2f} minutes.")

In [ ]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [ ]:
df_result = pd.concat([df['ride_id'], pd.Series(y_pred, name='prediction')], axis=1)

os.makedirs(output_dir, exist_ok=True)
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)